## Exercise 13: Voting

In this exercise we are exploring the difficulties in voting and aggregating the votes of a population. We will work with a set of 25 rankings of options A-G, which are loaded in the cell below.

In [81]:
import pickle

# list of voting options
opts = ['A','B','C','D','E','F','G','H']

# load list of rankings
rankings = pickle.load(open("rankings.pickle",'rb'))
rankings

[['F', 'H', 'D', 'B', 'A', 'E', 'C', 'G'],
 ['C', 'A', 'B', 'E', 'F', 'H', 'D', 'G'],
 ['C', 'G', 'B', 'E', 'A', 'F', 'H', 'D'],
 ['E', 'F', 'D', 'G', 'A', 'H', 'B', 'C'],
 ['G', 'A', 'E', 'D', 'C', 'H', 'F', 'B'],
 ['A', 'H', 'D', 'G', 'B', 'E', 'C', 'F'],
 ['B', 'C', 'G', 'H', 'A', 'F', 'E', 'D'],
 ['G', 'E', 'H', 'B', 'C', 'F', 'A', 'D'],
 ['C', 'B', 'D', 'G', 'E', 'H', 'A', 'F'],
 ['E', 'G', 'B', 'C', 'F', 'H', 'D', 'A'],
 ['D', 'G', 'F', 'C', 'B', 'E', 'A', 'H'],
 ['G', 'D', 'B', 'C', 'H', 'F', 'E', 'A'],
 ['D', 'H', 'C', 'F', 'B', 'G', 'E', 'A'],
 ['H', 'E', 'A', 'B', 'F', 'C', 'D', 'G'],
 ['F', 'H', 'D', 'E', 'A', 'C', 'G', 'B'],
 ['H', 'E', 'C', 'D', 'F', 'B', 'A', 'G'],
 ['H', 'A', 'B', 'F', 'C', 'E', 'G', 'D'],
 ['G', 'B', 'H', 'C', 'A', 'E', 'D', 'F'],
 ['E', 'A', 'F', 'G', 'B', 'H', 'C', 'D'],
 ['C', 'F', 'A', 'D', 'B', 'G', 'E', 'H'],
 ['D', 'E', 'F', 'G', 'H', 'B', 'A', 'C'],
 ['F', 'A', 'H', 'E', 'C', 'D', 'B', 'G'],
 ['F', 'C', 'G', 'H', 'A', 'B', 'E', 'D'],
 ['H', 'G',

### Task 1: The Majority Rule and Condorcet's Paradox

Write a function that impements the majority rule based on a set of rankings, using the signature in the cell below. Apply this function on the given ranking data, by computing the majority winner of all pairs of the ranked options. Do you observe Condorcet's paradox?

In [1]:
# INPUT PARAMETERS
# pair: tuple of 2 elements on which we want to determine the majority vote
# rankings: List of individual rankings
#
# return: winner of majority vote
def majority_vote(pair,rankings):
    
    def single_pref(alt1,alt2,ranking):
        for e in ranking:
            if e == alt1:
                return alt1
            if e == alt2:
                return alt2
    alt1, alt2 = pair[0],pair[1]
    
    votes = dict({alt1: 0,
                  alt2: 0})
    for ranking in rankings:
        votes[single_pref(alt1,alt2,ranking)] += 1
    
    return max(votes, key=votes.get)

In [14]:
majority_vote(('E','B'),rankings)

'E'

In [41]:
import itertools
pairs = set(itertools.combinations(opts, 2))
for pair in pairs:
    print(str(pair) + ": " + str(majority_vote(pair,rankings)))

('E', 'F'): E
('G', 'H'): G
('D', 'F'): F
('A', 'E'): E
('B', 'G'): G
('C', 'E'): E
('A', 'G'): G
('B', 'E'): B
('C', 'G'): C
('A', 'D'): A
('D', 'E'): E
('B', 'F'): F
('C', 'D'): C
('B', 'C'): B
('F', 'H'): H
('A', 'C'): C
('E', 'H'): H
('E', 'G'): G
('A', 'H'): H
('B', 'H'): H
('B', 'D'): D
('C', 'H'): H
('D', 'G'): D
('D', 'H'): H
('A', 'B'): B
('C', 'F'): C
('A', 'F'): F
('F', 'G'): F


__Answer:__ We observe Condorcet's paradox multiple times. For instance, G is preferred over H, and H is preferred over F, but F is preferred over G.

### Task 2: Voting Systems

Write two functions which implement the sequential and the tournament voting systems, using the signatures in the cell below.
For each system, simulate 1000 runs based on different permutations of the input list, and count the number of times each element wins (note that when ranking $n$ elements, there are $O(n!)$ possible sequential agendas).  
Does every option get to win once in sequential and tournament voting? Is there an element that dominates the votes? Which voting system would you generally prefer and why?

In [66]:
# INPUT PARAMETERS
# rankings: List of individual rankings
# seq: sequence specifying the agenda. first two elements are voted against each other, 
#      and then the winner is voted against the next element in the sequence, e.g. in 
#      [A,C,B,D], first A and B are voted against each other, then the winner is voted 
#      against C, and this winner is voted against D
#
# return: winner of sequential vote
def seq_vote(rankings, seq):
    sq = seq.copy()
    alt1 = sq.pop(0)
    alt2 = sq.pop(0)
    
    winner = majority_vote((alt1,alt2),rankings)
    while sq:
        winner = majority_vote((winner,sq.pop(0)),rankings)
        
    return winner

In [67]:
seq_vote(rankings,opts)

'G'

In [74]:
# INPUT PARAMETERS
# rankings: List of individual rankings
# tree: list specifying tournament tree, i.e., the lowest level of the tournament tree.
# Example: with tree = [A,C,B,D], in first round we have the votes A vs c and B vs D. In the next round,
# the winner of these are then voted against each other
# -> you may assume that there are always 2^i elements to rank
#
# return: winner of tournament vote
def tournament_vote(rankings, tree):
    curr_tree = tree.copy()
    next_tree = []
    while len(curr_tree)>2:
        while curr_tree:
            alt1 = curr_tree.pop(0)
            alt2 = curr_tree.pop(0)
            next_tree.append(majority_vote((alt1,alt2),rankings))
        
        curr_tree = next_tree.copy()
        next_tree = []
        
    return majority_vote((curr_tree[0],curr_tree[1]),rankings)

In [75]:
tournament_vote(rankings,opts)

'G'

In [87]:
import random
seq_counts = dict({key : 0 for key in opts})
for _ in range(1000):
    w = seq_vote(rankings,random.sample(opts, 8))
    seq_counts[w]+=1

print(seq_counts)

{'A': 6, 'B': 50, 'C': 118, 'D': 47, 'E': 76, 'F': 91, 'G': 221, 'H': 391}


In [88]:
t_counts = dict({key : 0 for key in opts})
for _ in range(1000):
    w = tournament_vote(rankings,random.sample(opts, 8))
    t_counts[w]+=1

print(t_counts)

{'A': 0, 'B': 12, 'C': 75, 'D': 0, 'E': 32, 'F': 50, 'G': 240, 'H': 591}


__Answer:__ In general, H seems to dominate noth vting systems, though this trend is much stronger for elimination tournaments. 
Elimination tournaments is arguably fairer, as in that setting it is not possible to win the tournament if by majority vote you only beat one out of all other options. 

### Task 3: Positional Voting

#### a) Borda Count

Write a function that implements the Borda count, using the signature in the cell below, which also allows for optional allocation of weights. Apply this function on the giving rankings, using
* standard weights, i.e., for a ranking of $k$ elements, the first gets $k$ points, the scond $k-t$ points, and so on
* top 5 weights, i.e., in every ranking only the top 5 get points, with 5 points for the first, 4 points for the second, etc
* Step-wise halving weights, i.e. the first element gets $2k$ points, the second $k$ points, the third $k/2$ points, etc

Are the resulting rankings identical, and do they correspond to what you would expect from your solution in task 2?

In [22]:
import numpy as np

# INPUT PARAMETERS:
# rankings: list of lists of individual rankings
# weights: list of k descending weights that are used to score/aggregate individual rankings.
# By default, the standard weights [k,k-1,k-2,...] should be used.
#
# return a list of (element,score) tuples, in descending order by given score
def borda_count(rankings, weights = None):
    keys = list(set(rankings[0]))
    scores = {key : 0 for key in keys}
    k = len(keys)
    
    if weights is None:
        weights = np.arange(k,0,step = -1)
    
    for ranking in rankings:
        for i in range(k):
            e = ranking[i]
            scores[e] += weights[i]
            
    return [(k,v) for k,v in sorted(scores.items(), key=lambda item: item[1], reverse=True)] 

In [89]:
pos_voting(rankings)

[('H', 122),
 ('E', 119),
 ('G', 118),
 ('C', 116),
 ('F', 111),
 ('B', 109),
 ('A', 106),
 ('D', 99)]

In [90]:
pos_voting(rankings, weights = [5,4,3,2,1,0,0,0])

[('G', 55),
 ('H', 52),
 ('C', 50),
 ('E', 49),
 ('F', 45),
 ('A', 42),
 ('B', 42),
 ('D', 40)]

In [91]:
pos_voting(rankings, weights = [16,8,4,2,1,.5,0.25,0.125])

[('G', 120.125),
 ('H', 118.5),
 ('E', 113.75),
 ('C', 110.125),
 ('F', 103.125),
 ('D', 85.25),
 ('A', 76.625),
 ('B', 69.375)]

__Answer:__ It becomes apparent that different weights yield diffrent rankings. Surprisingly, exect for the standard Borda count, these rankings do not display the trend towards H as the most popular element that we observed in task 2.

#### b) Arrow's Impossibility

Implement a function subrank, which, for a given single person's ranking, only extracts the relative ranking of a given subset of options. E.g., if we consider the first persons ranking (F, H, D, B, A, E, C, G), the subrank with respect to the subset {A,B,C} would be (B,A,C).  
Apply this function to grow the given set of rankings, i.e., initially you compute all subranks with respect to {A,B}, then all subranks with respect to {A,B,C}, and so on, until we have the original individual rankings.  
At each step, compute the Borda Count ranking over all current individual subranks. Do you observe a violation of the IIA assumption?

In [93]:
# INPUT PARAMETERS:
# ranking: list of an individual person's ranking
# elems: subset of all elements based on which we compute the subrank
#
# return a list which represents the current subrank
def subrank(ranking, opts):
    res = []
    for r in ranking:
        if r in opts:
            res.append(r)
    return res

In [95]:
for i in np.arange(2,9):
    curr_rankings = [subrank(r,opts[:i]) for r in rankings]
    print(pos_voting(curr_rankings))

[('B', 38), ('A', 37)]
[('B', 51), ('C', 51), ('A', 48)]
[('C', 66), ('B', 63), ('A', 63), ('D', 58)]
[('E', 80), ('C', 77), ('B', 77), ('A', 74), ('D', 67)]
[('E', 95), ('C', 91), ('B', 89), ('F', 86), ('A', 86), ('D', 78)]
[('E', 107), ('C', 105), ('G', 103), ('F', 99), ('B', 99), ('A', 96), ('D', 91)]
[('H', 122), ('E', 119), ('G', 118), ('C', 116), ('F', 111), ('B', 109), ('A', 106), ('D', 99)]


__Answer__: The IIA assumption is violated with respect to elements B and F, where initially B is ranked over F, but in the end their ranking is swapped. 